In [1]:
import pandas as pd
import numpy as np
import copy
import pickle 

pd.options.display.max_columns = 200
pd.options.display.max_columns = 200

In [2]:
fp_sales = '../data/sales.csv'
df_sales = pd.read_csv(fp_sales, encoding='ISO-8859-1', sep=',')

In [3]:
df_sales.head(1)

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018,1,2,2.0,44,10.0,6352.94,0.03,0.0


In [4]:
df_sales.isna().sum()

Cliente           0
Año               0
Mes               0
ClaseEnvase       0
SegmentoPrecio    5
Marca             0
Cupo              5
Nr                0
Hl                0
Dcto              0
dtype: int64

In [5]:
df_sales = df_sales.dropna()

In [6]:
#Convertir anio, mes a string
df_sales['Año'] = df_sales['Año'].astype(str)
df_sales['Mes'] = df_sales['Mes'].astype(str)
df_sales["Mes"] = df_sales['Mes'].str.zfill(width=2)
df_sales.head(1)

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018,01,2,2.0,44,10.0,6352.94,0.03,0.0


In [7]:
# Unir Anio, Mes en una columna datetime
df_sales['periodo'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(df_sales["Año"], df_sales.Mes)])
df_sales.head(5)

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,periodo
0,1,2018,01,2,2.0,44,10.0,6352.94,0.03,0.0,2018-01-01
1,1,2018,01,2,4.0,18,16.0,99130.50,0.45,0.0,2018-01-01
2,1,2018,06,2,1.0,2,6.0,11605.40,0.07,-2880.0,2018-06-01
3,1,2018,07,1,2.0,37,34.0,1944.01,0.01,0.0,2018-07-01
4,1,2018,08,2,1.0,2,16.0,16250.18,0.06,0.0,2018-08-01


In [8]:
fechas_sales = list(df_sales['periodo'].unique())
fechas_sales.sort()
fechas_sales

[numpy.datetime64('2018-01-01T00:00:00.000000000'),
 numpy.datetime64('2018-02-01T00:00:00.000000000'),
 numpy.datetime64('2018-03-01T00:00:00.000000000'),
 numpy.datetime64('2018-04-01T00:00:00.000000000'),
 numpy.datetime64('2018-05-01T00:00:00.000000000'),
 numpy.datetime64('2018-06-01T00:00:00.000000000'),
 numpy.datetime64('2018-07-01T00:00:00.000000000'),
 numpy.datetime64('2018-08-01T00:00:00.000000000'),
 numpy.datetime64('2018-09-01T00:00:00.000000000'),
 numpy.datetime64('2018-10-01T00:00:00.000000000'),
 numpy.datetime64('2018-11-01T00:00:00.000000000'),
 numpy.datetime64('2018-12-01T00:00:00.000000000'),
 numpy.datetime64('2019-01-01T00:00:00.000000000'),
 numpy.datetime64('2019-02-01T00:00:00.000000000'),
 numpy.datetime64('2019-03-01T00:00:00.000000000'),
 numpy.datetime64('2019-04-01T00:00:00.000000000'),
 numpy.datetime64('2019-05-01T00:00:00.000000000'),
 numpy.datetime64('2019-06-01T00:00:00.000000000'),
 numpy.datetime64('2019-07-01T00:00:00.000000000'),
 numpy.datet

In [9]:
# Modificar ventas de tal manera que un cliente siempre tenga una fila para todos los periodos
# Llenaremos Nr, Hl, Dcto con  0.0
# Esto es valido porque no estamos utilizando ClaseEnvase, SegmentoPrecio, MarcaCupo
l_unique_periodo = list(df_sales['periodo'].unique())
l_unique_cliente = list(df_sales['Cliente'].unique())


l_unique_periodo_fin = l_unique_periodo * len(l_unique_cliente)
l_unique_cliente_fin = [[cliente]*len(l_unique_periodo) for cliente in l_unique_cliente]
#flatten
l_unique_cliente_fin = [item for sublist in l_unique_cliente_fin for item in sublist]

#l_unique_cliente = [[cliente]*len(l_unique_periodo) for cliente in l_unique_cliente]

# Creamos un dataframe lienzo
df_sales_base = pd.DataFrame(
  list(zip(l_unique_periodo_fin, l_unique_cliente_fin)),
  columns = ['periodo', 'Cliente']
)
df_sales_base.head(1)

,periodo,Cliente
0,2018-01-01,1


In [10]:
# Revisar que tenemos todos los casos contemplados
#len(df_sales_complete) / (len(l_unique_periodo) * len(l_unique_cliente))

In [11]:
# A este lienzo le agregamos toda la info que existe verdaderamente

In [12]:
print(f"Longitud antes: {len(df_sales_base)}")
df_sales_complete = pd.merge( df_sales_base, df_sales, how='left', on=['periodo', 'Cliente'])
print(f"Longitud despues: {len(df_sales_complete)}")
df_sales_complete.head(1)

Longitud antes: 301380
Longitud despues: 1960141


,periodo,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,2018-01-01,1,2018,01,2.0,2.0,44.0,10.0,6352.94,0.03,0.0


In [13]:
df_sales_complete.isna().sum()

periodo               0
Cliente               0
Año               65894
Mes               65894
ClaseEnvase       65894
SegmentoPrecio    65894
Marca             65894
Cupo              65894
Nr                65894
Hl                65894
Dcto              65894
dtype: int64

In [14]:
df_sales_complete.dtypes

periodo           datetime64[ns]
Cliente                    int64
Año                       object
Mes                       object
ClaseEnvase              float64
SegmentoPrecio           float64
Marca                    float64
Cupo                     float64
Nr                       float64
Hl                       float64
Dcto                     float64
dtype: object

In [15]:
# Completamos nan
df_sales_complete['ClaseEnvase'] = df_sales_complete['ClaseEnvase'].fillna(99)
df_sales_complete['SegmentoPrecio'] = df_sales_complete['SegmentoPrecio'].fillna(99)
df_sales_complete['Marca'] = df_sales_complete['Marca'].fillna(99)
df_sales_complete['Cupo'] = df_sales_complete['Cupo'].fillna(99)

# df_sales_complete['Nr'] = df_sales_complete['Nr'].fillna(0)
# df_sales_complete['Hl'] = df_sales_complete['Hl'].fillna(0)
# df_sales_complete['Dcto'] = df_sales_complete['Dcto'].fillna(0)

In [16]:
df_sales_complete.head(1)

,periodo,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,2018-01-01,1,2018,01,2.0,2.0,44.0,10.0,6352.94,0.03,0.0


In [17]:
df_sales_complete.isna().sum()

periodo               0
Cliente               0
Año               65894
Mes               65894
ClaseEnvase           0
SegmentoPrecio        0
Marca                 0
Cupo                  0
Nr                65894
Hl                65894
Dcto              65894
dtype: int64

In [18]:
df_sales_complete.groupby('Cliente').agg({'Mes': 'count'})

,Mes
Cliente,
1,179
2,180
3,100
4,139
5,37
...,...
15065,39
15066,63
15067,22


In [19]:
# crear columna periodo y dropear las innecearias
# crear agrupados por (periodo, cliente) | otra granularidad, (periodo, cliente, Marca, Cupo)
# calcular columnas de sumas de N_r, Hl, Dcto
# para cada una de las columnas de suma calcular el rolling_mean/sum a X periodos en el pasado
# repertirlo para todos los periodos en el pasado que se deseen
# llenar los nan con interpolacion / media del grupo(nivel de granularidad)

def calcular_acumulados_pasado(df, lista_grupo, dict_columnas, modo='sum'):
  """
  Calcula agrupado del df(sales) hacia el pasado utilizando rolling mean y agrupando por lista_grupo.
  
  Input:
  ======
  df: pd.Dataframe: tabla de ventas(tal vez se podria generalizar este utilitario)
  lista_grupo: lista[str] : utilizados para generar los agrupados sobre los que se haran calculos
  dict_columnas: dict{ 'key='Nr'/'Hl'/'Dcto': [x(int), y(int), z(int)]  }
    [x, y, z] indica que se calculara el acumulado para la llave x , y, z meses en el pasado
  """
  
  # Una copia de la tabla de ventas para no incomodarnos con modificarla por error
  c_df = copy.deepcopy(df)
  
  # Creamos agrupado por lista_grupo y para las llaves en dict_columnas con np.sum como valor
  agg_dict = {k: [np.sum, 'count'] for k, _ in dict_columnas.items()}
  # por alguna razon este np.sum no considera los nans
  # y 'count' solo cuenta los no nan, justo lo que queremos
  # https://stackoverflow.com/questions/42770300/summing-rows-in-grouped-pandas-dataframe-and-return-nan
  
  df_g1 = c_df.groupby(lista_grupo).agg(agg_dict)
  # Completar nan con cero, estos NaN vienen de Nr, Hl, Dcto 

  
  # Agregamos multiples niveles en el nombre de las columnas
  df_g1.columns = ['_'.join(col) for col in df_g1.columns]# Nombre seran de la forma Nr_sum, Nr_count, ...
  df_g1 = df_g1.reset_index() 
  # Comenzamos a calcular los agrupados segun dict_columnas
  for col_name, months_past_list in dict_columnas.items():
    for window in months_past_list:
      if modo == 'sum':
        rol_col_name = modo + '_' + str(window) + col_name
        #quitamos periodo de lista grupo    
        df_g1[rol_col_name] = df_g1.sort_values('periodo', ascending=True)\
                                .groupby(lista_grupo[1:])\
                                [col_name+'_sum'].apply(lambda x: x.rolling(window).sum())
        #llenar nan de estos agrupados con la media del grupo
        df_g1[rol_col_name] = df_g1.groupby(lista_grupo[1:])\
                                [rol_col_name].transform(lambda x: x.fillna(x.mean())) 
                                
      elif modo == 'mean':
        rol_col_name = modo + '_' + str(window) + col_name
        df_g1[rol_col_name] = df_g1.sort_values('periodo', ascending=True)\
                                .groupby(lista_grupo[1:])\
                               [col_name + '_sum'].apply(lambda x: x.rolling(window).mean())
        #llenar nan de estos agrupados con la media del grupo
        df_g1[rol_col_name] = df_g1.groupby(lista_grupo[1:])\
                                [rol_col_name].transform(lambda x: x.fillna(x.mean())) 
      elif modo == 'todo':
        # suma
        rol_col_name = 'sum' + '_' + str(window) + col_name
        #quitamos periodo de lista grupo    
        df_g1[rol_col_name] = df_g1.sort_values('periodo', ascending=True)\
                                .groupby(lista_grupo[1:])\
                                [col_name+'_sum'].apply(lambda x: x.rolling(window).sum())
        #llenar nan de estos agrupados con la media del grupo
        df_g1[rol_col_name] = df_g1.groupby(lista_grupo[1:])\
                                [rol_col_name].transform(lambda x: x.fillna(x.mean()))    
        # media
        rol_col_name = 'mean' + '_' + str(window) + col_name
        df_g1[rol_col_name] = df_g1.sort_values('periodo', ascending=True)\
                                .groupby(lista_grupo[1:])\
                               [col_name + '_sum'].apply(lambda x: x.rolling(window).mean())
        #llenar nan de estos agrupados con la media del grupo
        df_g1[rol_col_name] = df_g1.groupby(lista_grupo[1:])\
                                [rol_col_name].transform(lambda x: x.fillna(x.mean()))     
      # PODRIA MEJORAR ESTO METIENDO TODO EN UN MISMO GROUPBY!
      

  return df_g1
     

In [20]:
def calcular_grupo_2(df, lista_grupo, dict_columnas, window_ventas_list, modo='todo'):
  # Agregamos periodo a partir de anio y mes
  # Convertir anio, mes a string
  df_tg0 = copy.deepcopy(df)
  # Dropear columnas innecesarias
  df_tg0 = df_tg0.drop(columns=['Mes', 'Año'], axis=1)  
  # Calculamos los acumulados segun Nr / Hl / Dcto
  df_tg1 = calcular_acumulados_pasado(df_tg0, lista_grupo, dict_columnas, modo=modo)
  # Agregar columna count que reemplaza a  Nr_count Hl_count Dcto_count
  cols_count = [k + '_count' for k, _ in test_dict.items()]
  df_tg1 = df_tg1.drop(columns = cols_count[1:], axis=1)
  df_tg1 = df_tg1.rename(columns={cols_count[0]: 'numero_ventas'})
  # Calcular acumulados de tiempo en el pasado
  df_tg2 = calcular_acumulados_pasado(df_tg1, lista_grupo, {'numero_ventas': window_ventas_list}, 
                                       modo = 'todo')
  # Eliminar numero_ventas_count
  df_tg2 = df_tg2.drop(columns = 'numero_ventas_count', axis = 1)
  # Juntar los dos acumulados
  df_tg_unido = pd.merge(df_tg1, df_tg2, on = lista_grupo, how = 'left')
  # agregar Hl/ Nr
  return df_tg_unido

In [21]:

t_list = ['periodo', 'Cliente']# tambien con Marca , Cupo
test_dict = {
  'Nr': [2,],
  'Hl': [2,],
  'Dcto': [2,]
}
window_ventas_list = [2,]
df_fin = calcular_grupo_2(df_sales_complete, t_list, test_dict, window_ventas_list, modo='todo')

In [22]:
df_fin.head(5)

,periodo,Cliente,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_2Hl,mean_2Hl,sum_2Dcto,mean_2Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas
0,2018-01-01,1,591975.69,13,2.31,-6785.54,680049.570526,340024.785263,2.491579,1.245789,-47691.214211,-23845.607105,13,17.736842,8.868421
1,2018-01-01,2,482188.20,9,1.30,-9811.80,581925.716842,290962.858421,1.784211,0.892105,-46216.870526,-23108.435263,9,18.473684,9.236842
2,2018-01-01,3,559180.83,13,2.06,-42516.22,321979.125789,160989.562895,1.338947,0.669474,-29634.394737,-14817.197368,13,9.842105,4.921053
3,2018-01-01,4,400454.04,6,1.77,-4747.57,487162.189474,243581.094737,2.202632,1.101316,-47444.200526,-23722.100263,6,14.315789,7.157895
4,2018-01-01,5,1263011.57,6,4.64,-10131.66,995558.801579,497779.400789,3.905263,1.952632,-41319.371579,-20659.685789,6,3.578947,1.789474


In [23]:
df_fin.isna().sum()

periodo                0
Cliente                0
Nr_sum                 0
numero_ventas          0
Hl_sum                 0
Dcto_sum               0
sum_2Nr                0
mean_2Nr               0
sum_2Hl                0
mean_2Hl               0
sum_2Dcto              0
mean_2Dcto             0
numero_ventas_sum      0
sum_2numero_ventas     0
mean_2numero_ventas    0
dtype: int64

In [24]:
len(df_fin)

301380

In [25]:
fp_output = '../tablones/data_grupo_3.pkl'
def store_pickle(fp, obj):
  with open(fp, 'wb') as f:
    pickle.dump(obj, f)
    
store_pickle(fp_output, df_fin)